## 1 Hierarchical Indexing

___Hierarchical indexing___ is an important fearture of pandas that enables you to have multiple index levels on an axis.

Create a Series with a list of lists (or arrays) as the index, `MultiIndex`

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [8]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 2, 3, 1, 2, 3]])

In [9]:
data

a  1    1.505454
   2   -0.115230
   3    1.244119
b  1   -0.043838
   2   -0.480653
c  3    0.910750
   1   -0.256528
d  2    1.548743
   3   -1.431489
dtype: float64

In [10]:
data.index

MultiIndex(levels=[[u'a', u'b', u'c', u'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data

In [11]:
data['b']

1   -0.043838
2   -0.480653
dtype: float64

In [12]:
data['b':'c']

b  1   -0.043838
   2   -0.480653
c  3    0.910750
   1   -0.256528
dtype: float64

In [13]:
data.loc[['b', 'd']]

b  1   -0.043838
   2   -0.480653
d  2    1.548743
   3   -1.431489
dtype: float64

In [14]:
data.loc[:, 2]

a   -0.115230
b   -0.480653
d    1.548743
dtype: float64

Rearrange the data into a DataFrame using its `unstack` method

In [16]:
data.unstack()

,1,2,3
a,1.505454,-0.115230,1.244119
b,-0.043838,-0.480653,NaN
c,-0.256528,NaN,0.910750
d,NaN,1.548743,-1.431489


The inverse operation of `unstack` is `stack`

In [17]:
data.unstack().stack()

a  1    1.505454
   2   -0.115230
   3    1.244119
b  1   -0.043838
   2   -0.480653
c  1   -0.256528
   3    0.910750
d  2    1.548743
   3   -1.431489
dtype: float64

With a DataFrame, either axis can have a hierarchical index

In [18]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']],
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

In [19]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names

In [20]:
frame.index.names = ['key1', 'key2']

In [21]:
frame.columns.names = ['state', 'color']

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Partial indexing of a DataFrame

In [23]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [24]:
frame.loc['a']

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5

In [25]:
frame.loc['a', 'Ohio']

color,Green,Red
key2,,
1,0,1
2,3,4


A `MultiIndex` can be created by itself and then reused

In [28]:
pd.MultiIndex(levels=[['Ohio', 'Colorado'], ['Green', 'Red']],
               labels=[[0, 0, 1], [0, 1, 0]],
               names=['state', 'color'])

MultiIndex(levels=[[u'Ohio', u'Colorado'], [u'Green', u'Red']],
           labels=[[0, 0, 1], [0, 1, 0]],
           names=[u'state', u'color'])

In [29]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex(levels=[[u'Colorado', u'Ohio'], [u'Green', u'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=[u'state', u'color'])

### 1.1 Reordering and Sorting Levels

The `swaplevel` method takes two level numbers or names and return a new object with the levels interchanged

In [34]:
frame.swaplevel('key2', 'key1')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [36]:
frame.swaplevel('state', 'color', axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

`sort_index` sorts the data using only the labels in a single level.

Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level

In [53]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

When sorting by a MultiIndex with `sort_values`, you need to contain tuple describing the column inside a list.

In [50]:
frame.sort_values([('Ohio', 'Green'), ('Colorado', 'Green')])

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 1.2 Summary Statitic by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you can specify the level you want to aggregate by on a particular axis

In [54]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [57]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 1.3 Indexing with a DataFrame's Columns

In [59]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                     'd': [0,1,2,0,1,2,3]})

In [60]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` method will create a new DataFrame using one or more of it's columns as the index

In [62]:
frame2 = frame.set_index(['c', 'd'])

In [63]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

`reset_index` does the opposite of set_index. The hierarchical index levels are moved into the columns

In [67]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


`stack` and `unstack` are for pivot table, `set_index` and `reset_index` are used to set/reset columns to index

## 2 Combining and Merging Datasets